In [ ]:
# Import necessary libraries
from QuantConnect import *
from QuantConnect.Data.Market import TradeBar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Initialize QuantBook
qb = QuantBook()


# Define assets
assets = {
    "EURUSD": qb.AddForex("EURUSD", Resolution.Daily, Market.FXCM).Symbol,
    "NVDA": qb.AddEquity("NVDA", Resolution.Daily).Symbol,
    "AMD": qb.AddEquity("AMD", Resolution.Daily).Symbol,
    "IEF": qb.AddEquity("IEF", Resolution.Daily).Symbol,
    "GLD": qb.AddEquity("GLD", Resolution.Daily).Symbol
}
# Define time period
start_date = datetime(2016, 1, 1)
end_date = datetime.now()
# Fetch historical data for all assets at once
symbols = list(assets.values())
history = qb.History(symbols, start_date, end_date, Resolution.Daily)

# Process historical data into a combined DataFrame
if not history.empty:
    # Initialize an empty DataFrame
    combined_data = pd.DataFrame()
    
    # Process each asset
    for name, symbol in assets.items():
        # Filter data for the current symbol
        symbol_data = history.loc[symbol][["close"]].rename(columns={"close": name})
        # Ensure datetime index is clean
        symbol_data.index = symbol_data.index.date
        # Add to combined DataFrame
        combined_data = combined_data.join(symbol_data, how="outer") if not combined_data.empty else symbol_data
    
    # Interpolate missing values
    combined_data = combined_data.interpolate()
    
    # Ensure index is datetime
    combined_data.index = pd.to_datetime(combined_data.index)
    
    # Set index name to "Date" to reflect daily data
    combined_data.index.name = "Date"
    
    print("Combined Asset DataFrame:")
    display(combined_data.head())
else:
    print("No historical data retrieved.")
# Store the DataFrame in Object Store
if not combined_data.empty:
    # Convert DataFrame to CSV string
    csv_data = combined_data.to_csv()
    # Save to Object Store
    qb.ObjectStore.Save("Asset_Data", csv_data)
    print("Asset data saved to Object Store under key 'Asset_Data'")



# Plot closing prices
if not combined_data.empty:
    plt.figure(figsize=(12, 6))
    for column in combined_data.columns:
        plt.plot(combined_data.index, combined_data[column], label=column)
    plt.title("Closing Prices of Assets (2016-2025)")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.grid(True)
    plt.show()


# Plot scaled closing prices
if not combined_data.empty:
    plt.figure(figsize=(12, 6))
    for column in combined_data.columns:
        plt.plot(combined_data.index, combined_data[column]/np.mean(combined_data[column]), label=column)
    plt.title("Closing Prices of Assets (2016-2025)")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.grid(True)
    plt.show()


# Save data to CSV for reference
if not combined_data.empty:
    combined_data.to_csv("asset_data.csv")
    print("Data saved to asset_data.csv")
